<a href="https://colab.research.google.com/github/AkersJack/TextToLatex/blob/main/Text2Latex_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install causal-conv1d
!pip install mamba-ssm
!pip install datasets
!pip install trl
!pip install peft
!pip install numba
!mkdir /content/data
!mkdir /content/mamba-chat


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 8.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cache

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
    drive.flush_and_unmount()
    #drive.mount('/content/drive', force_remount=True)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, MambaForCausalLM, MambaConfig
from mamba_ssm.models.mixer_seq_simple import MambaLMHeadModel

from datasets import load_dataset

from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from peft import LoraConfig
import pandas as pd

# from trainer.data import ChatDataModule
# from trainer.mamba_trainer import MambaTrainer

dataset = load_dataset("Kyudan/GTNT_PNT_WER")
dataset_chat = load_dataset("HuggingFaceH4/ultrachat_200k")

chatdataset = load_dataset("HuggingFaceH4/ultrachat_200k", split="train_sft")
chatdataset_val = load_dataset("HuggingFaceH4/ultrachat_200k", split="test_sft")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
roleDict_base = {
    "content":"", # Content here
    "role":"", # Role here (assistant or user)
}

textlist = []
data = dataset['train']

data = data.remove_columns("WER of NT PNT")

train_1 = data['NT']
train_2 = data['PNT']
key = data["GT"]
for x in range(0, len(train_1)):
  eq_list = []
  roleDict = roleDict_base.copy()
  roleDict["content"] = train_1[x]
  roleDict["role"] = "user"
  eq_list.append(roleDict)
  roleDict = roleDict_base.copy()
  roleDict["content"] = key[x]
  roleDict["role"] = "assistant"
  eq_list.append(roleDict)
  textlist.append(eq_list)
  eq_list = []
  roleDict = roleDict_base.copy()
  roleDict["content"] = train_2[x]
  roleDict["role"] = "user"
  eq_list.append(roleDict)
  roleDict = roleDict_base.copy()
  roleDict["content"] = key[x]
  roleDict["role"] = "assistant"
  eq_list.append(roleDict)
  textlist.append(eq_list)


messages_dataset = {"messages": textlist}
dataset_test = Dataset.from_dict(messages_dataset)

In [ ]:
import torch
import transformers
import json

from dataclasses import dataclass
from typing import Dict, Sequence
from tqdm import tqdm
from torch.utils.data import Dataset


class ChatDataset(Dataset):
    def __init__(self, data_path: str, tokenizer: transformers.AutoTokenizer, conversation_template: str, max_tokens: int):
        super(ChatDataset, self).__init__()
        data = []
        with open(data_path, "r") as file:
            for line in file:
                try:
                    data.append(json.loads(line))
                except Exception as e:
                    print("json processing exception", e)
                    continue


        data_dict = preprocess(data, tokenizer, conversation_template, max_tokens)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])


@dataclass
class DataCollatorForChatDataset(object):
    """
    Collate examples for supervised fine-tuning.
    """

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "input_ids"))
        input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id)
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=-100)

        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )


class ChatDataModule():
    def __init__(self, tokenizer: transformers.PreTrainedTokenizer, data_path: str, conversation_template, max_tokens: int):

        self.dataset = ChatDataset(tokenizer=tokenizer, data_path=data_path, conversation_template=conversation_template, max_tokens=max_tokens)
        self.data_collator = DataCollatorForChatDataset(tokenizer=tokenizer)


def preprocess(conversations: Sequence[Sequence[dict]], tokenizer: transformers.PreTrainedTokenizer, conversation_template: str, max_tokens: int) -> Dict:
    """
    Preprocess the data by tokenizing.
    """
    all_input_ids = []
    all_label_ids = []
    tokenizer.use_default_system_prompt = False

    print("Tokenizing dataset...")
    for conv in tqdm(conversations):
        current_conv = conv["messages"]
        tokenized_responses = []
        for msg in current_conv:
            if msg["role"] == "assistant":
                tokenized_responses.append(tokenizer.encode(msg["content"], add_special_tokens=False))

        tokenized_conv = tokenizer.apply_chat_template(current_conv, chat_template=conversation_template, max_length=max_tokens, truncation=True)
        all_input_ids.append(torch.LongTensor(tokenized_conv))


    return dict(input_ids=all_input_ids, labels=all_input_ids)

In [ ]:
with open("/content/data/math.jsonl", "w") as f:
    for d in dataset_test:
        f.write(json.dumps(dict(messages=d["messages"]))+"\n")

In [ ]:
data = dataset['train']

data = data.remove_columns("WER of NT PNT")

# GT key
# NT and PNT are 2 different ways to explain the equation

train_1 = data['NT']
train_2 = data['PNT']
key = data["GT"]

datad = {}

for x in range(0, len(train_1)):
  datad[train_1[x]] = key[x]
  datad[train_2[x]] = key[x]


input_list = list(datad.keys())

output_list = []
for x in input_list:
  output_list.append(datad[x])



dataf = pd.DataFrame({"Input":input_list, "Output":output_list})

dataf

#datasetm = Dataset.from_pandas(dataf)




,Input,Output
0,given the formula open curly bracket a to the ...,"Given the formula $[ \{ a^{ - } , a^{ + } \} ,..."
1,given the formula open curly bracket a to the...,"Given the formula $[ \{ a^{ - } , a^{ + } \} ,..."
2,when it comes to l c m of a g c d of b c equal...,When it comes to $\operatorname { l c m } ( a ...
3,when it comes to lcm of a gc div b c equals g...,When it comes to $\operatorname { l c m } ( a ...
4,when it comes to f sub u t equals f sub u minu...,"When it comes to $\operatorname { F }_{ u , t ..."
...,...,...
186843,in the context of the law the equation open p...,"In the context of the law, the equation $( { \..."
186844,when it comes to math bold i squared equals ma...,When it comes to $\mathbf { i }^{ 2 } = \mathb...
186845,when it comes to math bold i squared equals m...,When it comes to $\mathbf { i }^{ 2 } = \mathb...
186846,consider the equation delta g equals delta g n...,Consider the equation $\DeltaG = \DeltaG?? + R...


In [ ]:
from transformers import Trainer
import torch
import os


class MambaTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        input_ids = inputs.pop("input_ids")
        lm_logits = model(input_ids).logits

        labels = input_ids.to(lm_logits.device)
        shift_logits = lm_logits[:, :-1, :].contiguous()
        labels = labels[:, 1:].contiguous()

        loss_fct = torch.nn.CrossEntropyLoss()
        lm_loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), labels.view(-1))

        return lm_loss

    def save_model(self, output_dir, _internal_call):
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        #second_output_dir="/root/nextcloud/Documents/NLP_Project"
        torch.save(self.model.state_dict(), f"{output_dir}/pytorch_model.bin")
        #torch.save(self.model.state_dict(), f"{second_output_dir}/pytorch_model.bin")
        model.save_pretrained(output_dir)
        self.tokenizer.save_pretrained(output_dir)
        #self.tokenizer.save_pretrained(second_output_dir)

In [ ]:
#model_id = "havenhq/mamba-chat"
#model_id = "state-spaces/mamba-130m-hf"
#state-spaces/mamba-790m-hf
#model_id = "state-spaces/mamba-1.4b-hf"
model_id = "state-spaces/mamba-790m-hf"
#model_id2 = "state-spaces/mamba-2.8b-hf"
#tokenizer = AutoTokenizer.from_pretrained(model_id)
#model = MambaLMHeadModel.from_pretrained("state-spaces/mamba-1.4b-hf", device="cuda", dtype=torch.float16)

model = AutoModelForCausalLM.from_pretrained(model_id, residual_in_fp32 = False)
#model.config.to_json_file("/content/drive/MyDrive/NLP_Project/mamba-chat-4/checkpoint-2672/config.json")
#model2 = AutoModelForCausalLM.from_pretrained(model_id2)
#model = MambaLMHeadModel.from_pretrained("havenhq/mamba-chat", device="cuda", dtype=torch.float16)

#Train = MambaTrainer(model_id, datasetm)
#model = MambaLMHeadModel.from_pretrained("havenhq/mamba-chat", device="cuda", dtype=torch.float16)
#python train_mamba.py --model state-spaces/mamba-2.8b --tokenizer EleutherAI/gpt-neox-20b --learning_rate 5e-5 --batch_size 4 --data_path ./data/ultrachat_small.jsonl --num_epochs 3




In [ ]:


#model.load_state_dict(torch.load("pytorch_model.bin"))
#rng_state = torch.load("rng_state.pth")
#torch.set_rng_state(rng_state)
#model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/NLP_Project/mamba-chat-4/checkpoint-2672/")
#model = MambaLMHeadModel.from_pretrained("/content/drive/MyDrive/NLP_Project/mamba-chat-4/checkpoint-2672/py", dtype=torch.bfloat16, device="cuda")

#model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/NLP_Project/mamba-chat-4/checkpoint-2672/", residual_in_fp32 = False)

model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/NLP_Project/mamba-chat-math2/checkpoint-7800", residual_in_fp32 = False)


In [ ]:
import json
from datasets import load_dataset

data = load_dataset("HuggingFaceH4/ultrachat_200k", split="train_sft")


with open("/content/data/ultrachat.jsonl", "w") as f:
    for d in data:
        f.write(json.dumps(dict(messages=d["messages"]))+"\n")

In [ ]:

torch.cuda.empty_cache()

# from numba import cuda
# device = cuda.get_current_device()
# device.reset()

In [ ]:
#model = MambaLMHeadModel.from_pretrained(model_id, dtype=torch.bfloat16, device="CPU")
#model_id = "state-spaces/mamba-790m-hf"
#model_id = "state-spaces/mamba-2.8b"
#model = AutoModelForCausalLM.from_pretrained(model_id)
#model = MambaLMHeadModel.from_pretrained(model_id, device="cuda", dtype=torch.float16)
#model = MambaLMHeadModel.from_pretrained(model_id, dtype=torch.bfloat16, device="cuda")


#model = MambaLMHeadModel.from_pretrained("havenhq/mamba-chat", device="cuda", dtype=torch.float16)
#"EleutherAI/gpt-neo-1.3B"
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")
#tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")

tokenizer.eos_token = "<|endoftext|>"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.chat_template = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-beta").chat_template




/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
data_module_math = ChatDataModule(
        tokenizer=tokenizer,
        data_path="/content/data/math.jsonl",
        conversation_template=tokenizer.chat_template,
        max_tokens=2048
        #max_tokens=1024
    )


Tokenizing dataset...


100%|██████████| 187236/187236 [01:50<00:00, 1697.17it/s]


In [ ]:
#!rsync -r /root/nextcloud/Documents/NLP_Project/data_module.pkl /content/data_module_1.pkl



In [ ]:
import pickle
# dir(data_module)
# data_module.dataset
def save_data_module(data_module, filepath):
    with open(filepath, 'wb') as f:
        pickle.dump(data_module, f)

# Load ChatDataModule instance
def load_data_module(filepath):
    with open(filepath, 'rb') as f:
        data_module = pickle.load(f)
    return data_module

#save_data_module(data_module_math, '/content/drive/MyDrive/NLP_Project/data_module_math.pkl')




In [ ]:
data_module = load_data_module("/content/data_module_small.pkl")

In [ ]:
#data_module = load_data_module("/content/data_module.pkl")
#data_module = load_data_module("/content/drive/MyDrive/NLP_Project/data_module.pkl")
data_module_math = load_data_module("/content/drive/MyDrive/NLP_Project/data_module_math.pkl")


In [ ]:
data_module = ChatDataModule(
        tokenizer=tokenizer,
        data_path="/content/data/ultrachat.jsonl",
        conversation_template=tokenizer.chat_template,
        #max_tokens=2048
        max_tokens=1024
    )


In [ ]:
#tokenizer.save_pretrained("/content/data")

('/content/data/tokenizer_config.json',
 '/content/data/special_tokens_map.json',
 '/content/data/tokenizer.json')

In [ ]:
trainer = MambaTrainer(
    model=model,
    train_dataset=data_module_math.dataset,
    tokenizer=tokenizer,
    args=TrainingArguments(
        learning_rate= 5e-5,
        num_train_epochs= 1,
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 1,
        optim = "adamw_torch",
        logging_dir="/content/drive/MyDrive/NLP_Project/mamba-chat-math2",
        output_dir="/content/drive/MyDrive/NLP_Project/mamba-chat-math2",
        #logging_dir="/content/mamba-chat",
        #output_dir="/content/mamba-chat",
        logging_steps=20,
        save_steps=200,
        save_total_limit=5,
    ),
    data_collator=data_module_math.data_collator,
)
with open('/content/drive/MyDrive/NLP_Project/mamba-chat-math2/checkpoint-7800/trainer_state.json', 'r') as f:
    trainer_state = json.load(f)
# Set the trainer state
trainer.current_epoch = trainer_state['epoch']
trainer.global_step = trainer_state['global_step']
trainer.state.log_history = trainer_state['log_history']
trainer.state.best_metric = trainer_state['best_metric']
trainer.state.best_model_checkpoint = trainer_state['best_model_checkpoint']

# Load the model and optimizer state
#trainer.optimizer.load_state_dict(torch.load('/content/drive/MyDrive/NLP_Project/mamba-chat-math/checkpoint-600/optimizer.pt'))
#model.load_state_dict(trainer_state['model_state_dict'])
#trainer.optimizer.load_state_dict(trainer_state['optimizer_state_dict'])



# trainer.train()

# dir(model)

In [ ]:
#trainer._load_from_checkpoint("/content/drive/MyDrive/NLP_Project/mamba-chat-math2/checkpoint-1200")
trainer.train(resume_from_checkpoint="/content/drive/MyDrive/NLP_Project/mamba-chat-math2/checkpoint-7800")

Step,Training Loss
7820,0.462100
7840,0.452500
7860,0.463600
7880,0.465400
7900,0.447300
7920,0.416600
7940,0.483100
7960,0.455700
7980,0.440500
8000,0.464900


KeyboardInterrupt: 

In [ ]:
device = "cuda"
#tokenizer = AutoTokenizer.from_pretrained("havenhq/mamba-chat")
tokenizer.eos_token = "<|endoftext|>"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.chat_template = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-beta").chat_template

#model = MambaLMHeadModel.from_pretrained("havenhq/mamba-chat", device="cuda", dtype=torch.float16)

In [ ]:
dir(model)

In [ ]:
messages = []
while True:
    user_message = input("\nYour message: ")
    messages.append(dict(
        role="user",
        content=user_message
    ))

    input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to("cuda")

    out = model.generate(input_ids=input_ids, max_length=2000, temperature=0.9, top_p=0.7, eos_token_id=tokenizer.eos_token_id)

    decoded = tokenizer.batch_decode(out)
    messages.append(dict(
        role="assistant",
        content=decoded[0].split("<|assistant|>\n")[-1])
    )

    print("Model:", decoded[0].split("<|assistant|>\n")[-1])